In [1]:
%matplotlib inline
#import deepdish as dd
from matplotlib import pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import numpy as np
import brainiak.eventseg.event
from scipy import stats
from scipy.stats import norm, zscore, pearsonr
from scipy.signal import gaussian, convolve
from sklearn import decomposition
import seaborn as sns 

from easyesn.optimizers import GradientOptimizer
from easyesn import PredictionESN
from easyesn.optimizers import GridSearchOptimizer
from easyesn import helper as hlp

Using Numpy backend.


In [2]:
nostop = True
if nostop == True:
    embeddings=np.load("Exp1-NYT4-IGBA-embeddings.npy")
else:
    embeddings=np.load("Exp1-NYT4-IGBA-embeddings-stop.npy")
print(embeddings.shape)

embeddingsn = np.empty((embeddings.shape[0],embeddings.shape[1]))

(789, 100)


In [3]:
in_small = np.load('CH2020_50/in_small.npy')
out_small = np.load('CH2020_50/out_small.npy')

In [4]:
inputDataTesting=embeddings

numNode = 1000
vectorDim = 100


states1 = np.empty((0,numNode))
states_cumul = np.zeros((0,numNode))



In [ ]:
for res_seed in range(100):
# set the seed
    

    # set up the reservoirs
    # 
    np.random.seed(res_seed)
    esn1 = PredictionESN(n_input=vectorDim, n_output=vectorDim, n_reservoir=numNode, leakingRate=0.05, regressionParameters=[1e-2], solver="lsqr", feedback=False)

    # train the reervoir with a new seed
    esn1.fit(in_small, out_small, transientTime="Auto", verbose=1)


    #run the reservoir

    states1n = np.empty((0,numNode))

    prediction1,states1  = esn1.predict(inputDataTesting)

    states1 = states1.T

    states1 = states1[:,101:]

    states1n = np.empty((states1.shape[0],states1.shape[1]))
    
    if res_seed == 0:
        states_cumul = states1
    else:
        states_cumul = states_cumul + states1

    # do the segmentation
    seg_no = 10
    evS1 = brainiak.eventseg.event.EventSegment(n_events=seg_no)
    evS1.fit(states1)

    if nostop == True:
        g = [74, 137, 197,  278, 424, 583, 698]
    else:
        g = [131, 233, 337, 470, 706, 939, 1126]
    ground = np.array(g)


    boundsS1 = np.where(np.diff(np.argmax(evS1.segments_[0], axis=1)))[0]
    nTR=embeddings.shape[0]

    bounds = boundsS1
    human_bounds = ground
    np.random.seed(0)

    event_counts = np.diff(np.concatenate(([0],bounds,[nTR])))
    nPerm = 1000
    perm_bounds = bounds
    threshold = 3

    match = np.zeros(nPerm+1)
    for p in range(nPerm+1):
        for hb in human_bounds:
            # check if match 
            if np.any(np.abs(perm_bounds - hb) <= threshold):
                match[p] += 1
        match[p] /= len(human_bounds)
        perm_counts = np.random.permutation(event_counts)
        perm_bounds = np.cumsum(perm_counts)[:-1]

    print(res_seed,'p = ' + str(norm.sf((match[0]-match[1:].mean())/match[1:].std())))

100% (9603 of 9603) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


0 p = 0.3583656066195882


100% (9615 of 9615) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


1 p = 0.8055572940961174


100% (9606 of 9606) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


...

100% (9627 of 9627) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


30 p = 0.030367404419270938


100% (9598 of 9598) |####################| Elapsed Time: 0:00:08 Time:  0:00:08


31 p = 0.41037031475397123


N/A% (0 of 9608) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

In [8]:
seg_no = 10
evS1 = brainiak.eventseg.event.EventSegment(n_events=seg_no)
evS1.fit(states_cumul)
    
boundsS1 = np.where(np.diff(np.argmax(evS1.segments_[0], axis=1)))[0]
nTR=embeddings.shape[0]

bounds = boundsS1
human_bounds = ground+1
np.random.seed(0)

event_counts = np.diff(np.concatenate(([0],bounds,[nTR])))
nPerm = 1000
perm_bounds = bounds
threshold = 3

match = np.zeros(nPerm+1)
for p in range(nPerm+1):
    for hb in human_bounds:
        # check if match 
        if np.any(np.abs(perm_bounds - hb) <= threshold):
            match[p] += 1
    match[p] /= len(human_bounds)
    perm_counts = np.random.permutation(event_counts)
    perm_bounds = np.cumsum(perm_counts)[:-1]

print(res_seed,'p = ' + str(norm.sf((match[0]-match[1:].mean())/match[1:].std())))

4 p = 0.3869683903549704
